<a href="https://colab.research.google.com/github/DuckShyamalan/COMP6248-Reproducibility-Challenge/blob/main/Sleep_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 5.2MB/s 


In [2]:
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torchvision.datasets import MNIST

# fix random seed for reproducibility
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import numpy as np
np.random.seed(seed)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# flatten 28*28 images to a 784 vector for each image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert to tensor
    transforms.Lambda(lambda x: x.view(-1))  # flatten into vector
])


trainset = MNIST(".", train=True, download=True, transform=transform)
testset = MNIST(".", train=False, download=True, transform=transform)

trainset.data = trainset.data[0:27105]
trainset.targets = trainset.targets[0:27105]

trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=True)

(len(trainloader.dataset))



Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


27105

In [3]:
class ANN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, hidden_size)  
        self.fc3 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = F.dropout(out,p=0.2)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        
        if not self.training:
            out = F.softmax(out, dim=1)
        return out

# build the model
model = ANN(784, 1200, 10)

In [5]:
import torchbearer
# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()
optimiser = optim.SGD(model.parameters(), lr=0.1, momentum=0.5)

# Construct a trial object with the model, optimiser and loss.
# Also specify metrics we wish to compute.
trial = torchbearer.Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)

# Provide the data to the trial
trial.with_generators(trainloader, test_generator=testloader)

# Run 10 epochs of training
trial.run(epochs=2)

# test the performance
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)


{'test_loss': 1.567265272140503, 'test_acc': 0.9358000159263611}


In [6]:
torch.save(model.state_dict(),"ann_model.pt")

In [ ]:
print(model.fc1.weight)
print(model.fc2.weight)
print(model.fc3.weight)

In [7]:
def convertANNtoSNN(model):
    
    W1 = torch.clone(model.fc1.weight.data)
    W2 = torch.clone(model.fc2.weight.data)
    W3 = torch.clone(model.fc3.weight.data)
    
    SNN = ANN(W1.shape[1],W1.shape[0],W3.shape[0])
    
    SNN.fc1.weight.data = W1
    SNN.fc2.weight.data = W2
    SNN.fc3.weight.data = W3
    
    return SNN

In [8]:
SNN = convertANNtoSNN(model)

In [13]:
def sleep(SNN,inputs):
  
  maxFiringRate = 40.0
  current = 2.19

  W1 = torch.clone(SNN.fc1.weight).to(device)  #if .to(device) doesn't work, maybe try .cuda(device)?
  W2 = torch.clone(SNN.fc2.weight).to(device)
  W3 = torch.clone(SNN.fc3.weight).to(device)

  SNN.fc1.weight = torch.nn.Parameter(W1*current)
  SNN.fc2.weight = torch.nn.Parameter(W2*current)
  SNN.fc3.weight = torch.nn.Parameter(W3*current)


    
  voltages_1 = torch.zeros(1200,1).to(device)
  voltages_2 = torch.zeros(1200,1).to(device)
  voltages_3 = torch.zeros(10,1).to(device)
    
    
    
  for i in range(inputs.shape[0]):
        
    this_input = torch.reshape(inputs[i],(inputs.shape[1],1)).to(device)
        
    firingRates = this_input * maxFiringRate # list of firing rates for each feature
    firingRatesPer10ms = firingRates / 100 # expected spikes per 10ms per feature
        
    spike_trains = torch.rand(this_input.shape[0],100).to(device)
        
    
        
    spike_trains = spike_trains < firingRatesPer10ms
        
    spike_trains = spike_trains.float().to(device)
        
        
        
        
    # propogate spike trains through snn:
        
    for t in range(100):
      torch.cuda.empty_cache() 
      
            
      spike_trains_t = spike_trains[:,t].to(device)
      spike_trains_t = torch.reshape(spike_trains_t, (spike_trains_t.shape[0],1))
            
      # pass through input layer to first hidden layer:
            
      W1 = torch.clone(SNN.fc1.weight).to(device)
            
      voltages_1 = voltages_1 + (W1 @  spike_trains_t)

            
      x = voltages_1 > 0.03618
            
      spike_trains_t1 = x.float().to(device)
            
      W1 = W1 + (0.063 * (spike_trains_t1 @ spike_trains_t.T))
      W1 = W1 - (0.069 * (spike_trains_t1 @ (1-spike_trains_t).T))
            
            
            
      # pass through first hidden layer to second hidden layer:
            
      W2 = torch.clone(SNN.fc2.weight).to(device)
            
      
      voltages_2 = voltages_2 + (W2 @  spike_trains_t1)
        

      y = voltages_2 > 0.02336
            
      spike_trains_t2 = y.float().to(device)
            
      W2 = W2 + (0.063 * (spike_trains_t2 @ spike_trains_t1.T))
      W2 = W2 - (0.069 * (spike_trains_t2 @ (1-spike_trains_t1).T))
            
      # pass through second hidden layer to output layer:
            
      W3 = torch.clone(SNN.fc3.weight).to(device)
            
      voltages_3 = voltages_3 + (W3 @  spike_trains_t2)
            
      z = voltages_3 > 0.03638
            
      spike_trains_t3 = z.float().to(device)

      
      
            
      W3 = W3 + (0.063 * (spike_trains_t3 @ spike_trains_t2.T))
      W3 = W3 - (0.069 * (spike_trains_t3 @ (1-spike_trains_t2).T))

      
            
      SNN.fc1.weight = torch.nn.Parameter(W1)
      SNN.fc2.weight = torch.nn.Parameter(W2)
      SNN.fc3.weight = torch.nn.Parameter(W3)

      voltages_1 = voltages_1 - (voltages_1 * spike_trains_t1)
      voltages_2 = voltages_2 - (voltages_2 * spike_trains_t2)
      voltages_3 = voltages_3 - (voltages_3 * spike_trains_t3)

      #print(spike_trains_t1)
      #print("-----------------------")
      #print(spike_trains_t2)
      #print("-----------------------")
      #print(spike_trains_t3)
      #print("-----------------------")

  W1 = torch.clone(SNN.fc1.weight.data).to(device)
  W2 = torch.clone(SNN.fc2.weight.data).to(device)
  W3 = torch.clone(SNN.fc3.weight.data).to(device)

  SNN.fc1.weight = torch.nn.Parameter(W1/current)
  SNN.fc2.weight = torch.nn.Parameter(W2/current)
  SNN.fc3.weight = torch.nn.Parameter(W3/current)

       



        
      
        
        


In [14]:
with torch.no_grad():
  for data in trainloader:
      
      inputs = data[0]
      
      sleep(SNN,inputs)

KeyboardInterrupt: ignored

In [ ]:
print(SNN.fc1.weight.data.shape)
print(SNN.fc2.weight.data.shape)
print(SNN.fc3.weight.data.shape)

In [15]:
torch.save(SNN.state_dict(),"snn_model.pt")

In [16]:
from google.colab import files
files.download('snn_model.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
garbage stuff here to make the program stop running after it downloads

# New section